In [ ]:
!python -m pip install tensorflow==2.12.0 edgeimpulse

  Using cached tensorflow-2.12.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.4 kB)
  Using cached edgeimpulse-1.0.18-py3-none-any.whl.metadata (2.6 kB)
Using cached tensorflow-2.12.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (586.0 MB)
Using cached edgeimpulse-1.0.18-py3-none-any.whl (68 kB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-text 2.18.1 requires tensorflow<2.19,>=2.18.0, but you have tensorflow 2.12.0 which is incompatible.
tf-keras 2.18.0 requires tensorflow<2.19,>=2.18, but you have tensorflow 2.12.0 which is incompatible.


In [ ]:
import zipfile
import os

zip_path = "/content/Trash-Identification.zip"
extract_path = "/content/trash-identification"

# Extract ZIP file
if os.path.exists(zip_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    print("Dataset extracted successfully!")
else:
    print("File not found! Make sure the ZIP file is uploaded to /content/")


Dataset extracted successfully!


In [ ]:
train_dataset_path = "/content/trash-identification/Trash-Identification/train_dataset"
test_dataset_path = "/content/trash-identification/Trash-Identification/test_dataset"


In [ ]:
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import edgeimpulse as ei

import os
from PIL import Image

In [ ]:

# Function to verify images and remove corrupted files
def verify_images(directory):
    for root, _, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(root, file)
            try:
                with Image.open(file_path) as img:
                    img.verify()  # Verify if the image can be opened
            except (IOError, SyntaxError) as e:
                print(f"Removing corrupted or unsupported file: {file_path}")
                os.remove(file_path)

In [ ]:
# Verify images in the dataset
verify_images(train_dataset_path)
verify_images(test_dataset_path)

Removing corrupted or unsupported file: /content/trash-identification/Trash-Identification/train_dataset/Paper/53.jpg
Removing corrupted or unsupported file: /content/trash-identification/Trash-Identification/train_dataset/Paper/86.jpg


In [ ]:
# Set the Edge Impulse API key
ei.API_KEY = "ei_f3438badece0a852de243c2d50f16f745ec3a835fe520cb6b900024cd3ef9d28"


In [ ]:
# Labels for the dataset
labels = ["Plastic", "Metal Can", "Paper", "Rotten Fruits", "E-waste", "Water Bottle"]
num_classes = len(labels)

In [ ]:
# Filename for the exported model
deploy_filename = "my_model_cpp.zip"

In [ ]:
# Data preprocessing with ImageDataGenerator
datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,  # Normalize pixel values to [0, 1]
    validation_split=0.2  # Split dataset into 80% training and 20% validation
)

In [ ]:
# Load training and validation datasets
image_size = (64, 64)  # Resize images to 64x64
batch_size = 32

train_generator = datagen.flow_from_directory(
    train_dataset_path,
    target_size=image_size,
    batch_size=batch_size,
    class_mode="categorical",
    subset="training"
)

validation_generator = datagen.flow_from_directory(
    train_dataset_path,
    target_size=image_size,
    batch_size=batch_size,
    class_mode="categorical",
    subset="validation"
)

Found 743 images belonging to 6 classes.
Found 184 images belonging to 6 classes.


In [ ]:
# Testing dataset (Assumes a separate test dataset directory is provided)
test_datagen = ImageDataGenerator(rescale=1.0 / 255.0)  # Only rescaling for testing
test_generator = test_datagen.flow_from_directory(
    test_dataset_path,  # Path to the test dataset directory
    target_size=image_size,
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=False  # No shuffling to maintain consistent batch order
)

Found 230 images belonging to 6 classes.


In [ ]:
# Define the input shape
input_shape = (*image_size, 3)  # 3 channels for RGB

In [ ]:
# Build the model
model = keras.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation="relu", input_shape=input_shape),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(64, (3, 3), activation="relu"),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dense(num_classes, activation="softmax")
])

In [ ]:
# Compile the model
model.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=["accuracy"])

In [ ]:
# Train the model
model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=10
)

Epoch 1/10
16/24 [===================>..........] - ETA: 4s - loss: 1.9815 - accuracy: 0.2441

/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


24/24 [==============================] - 15s 582ms/step - loss: 1.8526 - accuracy: 0.2813 - val_loss: 1.3138 - val_accuracy: 0.5543
Epoch 2/10
24/24 [==============================] - 19s 807ms/step - loss: 1.1297 - accuracy: 0.5868 - val_loss: 0.8965 - val_accuracy: 0.6793
Epoch 3/10
24/24 [==============================] - 14s 565ms/step - loss: 0.8348 - accuracy: 0.7160 - val_loss: 0.6354 - val_accuracy: 0.8098
Epoch 4/10
24/24 [==============================] - 14s 563ms/step - loss: 0.6209 - accuracy: 0.8048 - val_loss: 0.5973 - val_accuracy: 0.7880
Epoch 5/10
24/24 [==============================] - 14s 568ms/step - loss: 0.4493 - accuracy: 0.8694 - val_loss: 0.6014 - val_accuracy: 0.8098
Epoch 6/10
24/24 [==============================] - 14s 572ms/step - loss: 0.3486 - accuracy: 0.8977 - val_loss: 0.6187 - val_accuracy: 0.7989
Epoch 7/10
24/24 [==============================] - 14s 563ms/step - loss: 0.2597 - accuracy: 0.9233 - val_loss: 0.5430 - val_accuracy: 0.8641
Epoch 8/10

In [ ]:
# Evaluate the model on the validation set
score = model.evaluate(validation_generator, verbose=0)
print(f"Validation loss: {score[0]}")
print(f"Validation accuracy: {score[1]}")


Validation loss: 0.6537488698959351
Validation accuracy: 0.8478260636329651


In [ ]:
# Evaluate model on test set
score = model.evaluate(test_generator, verbose=0)
print(f"Test loss: {score[0]}")
print(f"Test accuracy: {score[1]}")

Test loss: 1.0061217546463013
Test accuracy: 0.7782608866691589


In [ ]:
# List the available profile target devices
ei.model.list_profile_devices()

['alif-he',
 'alif-hp',
 'ambiq-apollo4',
 'arduino-nano-33-ble',
 'arduino-nicla-vision',
 'arduino-nicla-vision-m4',
 'portenta-h7',
 'brainchip-akd1000',
 'brickml',
 'cortex-m4f-80mhz',
 'cortex-m7-216mhz',
 'nxp-imx93-npu',
 'espressif-esp32',
 'himax-we-i',
 'himax-wiseeye2',
 'himax-wiseeye2-ethos',
 'imdt-v2h-cpu',
 'imdt-v2h',
 'infineon-cy8ckit-062s2',
 'infineon-cy8ckit-062-ble',
 'mbp-16-2020',
 'memryx-mx3',
 'microchip-sama7g54',
 'nordic-nrf52840-dk',
 'nordic-nrf5340-dk',
 'nordic-nrf9151-dk',
 'nordic-nrf9160-dk',
 'nordic-nrf9161-dk',
 'jetson-nano',
 'jetson-orin-nx',
 'jetson-orin-nano',
 'openmv-h7p',
 'particle-boron',
 'particle-p2',
 'raspberry-pi-4',
 'raspberry-pi-5',
 'raspberry-pi-rp2040',
 'renesas-ck-ra6m5',
 'renesas-ek-ra8d1',
 'renesas-rzg2l',
 'renesas-rzv2h-cpu',
 'renesas-rzv2h',
 'renesas-rzv2l-cpu',
 'renesas-rzv2l',
 'st-iot-discovery-kit',
 'st-stm32n6',
 'seeed-sense-cap',
 'wio-terminal',
 'seeed-vision-ai',
 'silabs-xg24',
 'silabs-thunderboar

In [ ]:
# Profile your model for target hardware
try:
    profile = ei.model.profile(model=model, device="arduino-nano-33-ble")
    print(profile.summary())
except Exception as e:
    print(f"Could not profile: {e}")

Target results for float32:
{
    "variant": "float32",
    "device": "arduino-nano-33-ble",
    "tfliteFileSizeBytes": 6507112,
    "isSupportedOnMcu": true,
    "memory": {
        "tflite": {
            "ram": 742198,
            "rom": 6554264,
            "arenaSize": 741894
        },
        "eon": {
            "ram": 616880,
            "rom": 6526784,
            "arenaSize": 615520
        },
        "eonRamOptimized": {
            "ram": 259760,
            "rom": 6536504,
            "arenaSize": 257088
        }
    },
    "timePerInferenceMs": 17226,
    "customMetrics": [],
    "hasPerformance": true
}


Performance on device types:
{
    "variant": "float32",
    "lowEndMcu": {
        "description": "Estimate for a Cortex-M0+ or similar, running at 40MHz",
        "timePerInferenceMs": 43142,
        "memory": {
            "tflite": {
                "ram": 742044,
                "rom": 6542488
            },
            "eon": {
                "ram": 616784,
   

In [ ]:
# List available deployment targets
ei.model.list_deployment_targets()

['syntiant-ndp101-lib',
 'syntiant-ndp120-lib',
 'syntiant-ndp120-lib-tdk-v14',
 'syntiant-ndp120-lib-ndp-v1-15-0',
 'syntiant-ndp101',
 'syntiant-nicla-ndp120',
 'syntiant-avnet-rasyn']

In [ ]:
# Set model information, such as your list of labels
model_output_type = ei.model.output_type.Classification(labels=labels)


In [ ]:
# Set model input type
model_input_type = ei.model.input_type.ImageInput()


In [ ]:

# Deploy the model as a C++ library
deploy_bytes = None
try:
    deploy_bytes = ei.model.deploy(
        model=model,
        model_output_type=model_output_type,
        model_input_type=model_input_type,
        deploy_target="zip"
    )
except Exception as e:
    print(f"Could not deploy: {e}")

In [ ]:
# Save the deployed model as a .zip file
if deploy_bytes:
    with open(deploy_filename, "wb") as f:
        f.write(deploy_bytes.getvalue())
    print(f"Model deployed successfully! Saved as {deploy_filename}.")

Model deployed successfully! Saved as my_model_cpp.zip.


In [ ]:
model.save("saved_model")  # Save in TensorFlow's SavedModel format
!zip -r saved_model.zip saved_model/  # Zip it for uploading


  adding: saved_model/ (stored 0%)
  adding: saved_model/assets/ (stored 0%)
  adding: saved_model/fingerprint.pb (stored 0%)
  adding: saved_model/variables/ (stored 0%)
  adding: saved_model/variables/variables.index (deflated 62%)
  adding: saved_model/variables/variables.data-00000-of-00001 (deflated 10%)
  adding: saved_model/saved_model.pb (deflated 88%)
  adding: saved_model/keras_metadata.pb (deflated 90%)
